### 7-1. 位相推定に基づくアルゴリズム

この章では位相推定 (Quantum Phase Estimation) のサブルーサブルーチンを用いた実用的なアルゴリズムを2例紹介する。以下で見るように、位相推定はユニタリー演算子の固有値問題に帰着できる問題に適用することができる。
1. 量子多体系ハミルトニアンの固有値問題 (水素分子の規定エネルギー)
2. 量子線形代数の逆行列を求める問題 (HHL アルゴリズム)

応用例を見る前に、位相推定アルゴリズムについて再度大まかな説明を行う。

### 7-1-1. 位相推定再訪
量子位相推定 (Quantum Phase Estimation, QPE) は、適当なユニタリー $U$ が与えられたとき、その固有値 $e^{i \lambda}$ をビット列として取り出すためのアルゴリズムである。

$U$ の固有値 $e^{i \lambda_i}$ に対応する固有ベクトルを $| u_i \rangle$ とする ($U | u_i \rangle = e^{i \lambda_i} | u_i \rangle$)。この時、制御ユニタリ演算 $\Lambda (U)$ と量子フーリエ変換を用いて、次の変換を実現する。
$$
| u_i \rangle | 0^{\otimes t} \rangle \xrightarrow{\text{QPE}} | u_i \rangle | \tilde{\lambda_i} \rangle
$$
ただし $\tilde{\lambda_i}$ は $\lambda_i$ を2進展開したビット列:
$$
\frac{\tilde{\lambda_i}}{2 \pi} = \frac{j_1}{2} + \frac{j_2}{2^2} + \ldots + \frac{j_t}{2^t} = 0.j_1 j_2 \ldots j_t
$$ 
である (詳細は[2-4. 位相推定アルゴリズム（入門編）](https://github.com/qulacs/quantum-native-dojo/blob/master/notebooks/2.4_phase_estimation_beginner.ipynb) を参照)。

この位相推定アルゴリズムをサブルーチンとして用いると、素因数分解や量子多体系のエネルギー計算といった固有値問題に帰着できる多くの問題を、古典コンピュータと比べて効率よく (指数的に高速) 解けることが期待されている事にも触れた。

ここで注意が必要なのは、このアルゴリズムは任意の制御ユニタリーゲート $\Lambda (U^{2k})$ ($0 \leq k \leq t$) が用意できると仮定している事である。実際、後で述べるように制御ユニタリーゲートを実機で実行可能なゲートセットに分解して構成したり、固有値のビット列を取り出す際の逆フーリエ変換を実行するには、一般的に量子回路が深くなってしまい長時間コヒーレント状態を維持する必要がある。特に測定精度の向上のためにビット数を増やす ($t$ を増やす) と、必要な測定時間は $\tau = 2^t$ の関係で増える。これは、フーリエ変換の $\Delta f \Delta \tau \sim 1$ の関係からも直感的にお分かり頂けるだろう。そのため誤り訂正機能を備えていないNISQ デバイスでは実現不可能と考えられている。

最後の逆フーリエ変換で用いられている制御位相ゲート $\Lambda (R_l)$ と、制御ビットの測定と測定結果 $m_k$ を用いて位相ゲートをかける操作 ($R^{m}_l$) は可換である事を用いて、必要な補助ビットを削減する方法も存在する。

反復的位相推定法 (Iterative Quantum Phase Estimation, IQPE) は固有値を2進展開したビット列を、各桁ごとに決定的に求める方法である。必要な補助ビットは 1つで、1イテレーションごとに1つの桁の値 ($j_k$) を求める。
手順は以下の通りである ($k = t, t-1, \ldots, 1$):

#### $k = t$ の時
1. 補助ビットに $H$ ゲートをかける
$$
| u_i \rangle | 0 \rangle \xrightarrow{H} | u_i \rangle | + \rangle 
$$
2. 補助ビットに $\Lambda (U^{2(t-1)})$ をかける
$$
| u_i \rangle | + \rangle \xrightarrow{\Lambda (U^{2t})} | u_i \rangle (| 0 \rangle + e^{-i \pi j_t} | 1 \rangle)
$$
3. 補助ビットに $H$ ゲートをかけて測定する
$$
| u_i \rangle (| 0 \rangle + e^{-i \pi j_t} | 1 \rangle) \xrightarrow{H} | u_i \rangle [(1 + e^{-i \pi j_t})| 0 \rangle + (1 - e^{-i \pi j_t}) | 1 \rangle)] \xrightarrow{\textrm{Measure}} | u_i \rangle | j_t \rangle
$$
4. 測定結果 $j_t$ を $\Phi(t)$ 反映させる: $\Phi(t) = j_t$

#### $k = t-1, t-2, \ldots, 1$ の時
1. 補助ビットに $H$ ゲートをかける
$$
| u_i \rangle | 0 \rangle \xrightarrow{H} | u_i \rangle | + \rangle 
$$
2. 補助ビットに $R_Z (\pi \Phi(k+1)/2)$ ($\Phi(k+1)/2 = 0.j_{k+1} j_{k+2} \ldots j_{t}$) をかける
$$
| u_i \rangle | + \rangle \xrightarrow{H} | u_i \rangle (| 0 \rangle + e^{+i \pi 0.j_{k+1} j_{k+2} \ldots j_{t}} | 1 \rangle)
$$
3. 補助ビットに $\Lambda (U^{2(k-1)})$ をかける
$$
| u_i \rangle (| 0 \rangle + e^{+i \pi 0.j_{k+1} j_{k+2} \ldots j_{t}} | 1 \rangle) \xrightarrow{\Lambda (U^{2(k-1)})} | u_i \rangle (| 0 \rangle + e^{-i \pi j_k} | 1 \rangle)
$$
4. 補助ビットに $H$ ゲートをかけて測定する
$$
| u_i \rangle (| 0 \rangle + e^{-i \pi j_k} | 1 \rangle) \xrightarrow{H} | u_i \rangle [(1 + e^{-i \pi j_k})| 0 \rangle + (1 - e^{-i \pi j_k}) | 1 \rangle)] \xrightarrow{\textrm{Measure}} | u_i \rangle | j_k \rangle
$$
5. 測定結果 $j_k$ を $\Phi(k)$ に反映させる: 
$$
\Phi(k) = \Phi(k+1)/2 + j_k = j_k.j_{k+1} j_{k+2} \ldots j_{t}
$$

#### $j_k$ ($k = t, t-1, \ldots, 1$) 測定後
$$
\lambda = 2 \pi \Phi(1)/2 = \pi 0.j_1.j_2 \ldots j_{t}
$$

このように逐次補助ビットにかける位相調節する事で、必要な補助ビットを1つに抑えることができる。次の節では反復的位相推定法を用いた水素分子の基底エネルギー測定方法を説明する。

### 7-1-2. 実用例： 位相推定を使った分子の基底状態エネルギーの計算

この節では "Scalable Quantum Simulation of Molecular Energies[1]" を参考に、位相推定サブルーチンを用いて水素分子の基底状態を求める計算を、実際に量子回路を構成して行う。
一般に量子化学計算では認めらる計算誤差が `chemical accuracy`( `$1.6 × 10^{−3}$ hartree`) という精度内に収まることが求められる。
本手法では、ハミルトニアンの固有値問題を、系のハミルトニアン $H$ の時間発展演算子 $U = e^{−iH \tau}$ の固有値を求めることで固有エネルギー $E_n$ を解く。
$|n \rangle$ をハミルトニアンの固有状態、それに対応する固有値を $E_n$ 、参照状態を $|\phi \rangle = \sum_n c_n |n \rangle$ とすると、
$$
e^{−iH \tau} |\phi \rangle = \sum_n e^{−i E_n \tau} c_n |n \rangle 
$$

この状態を補助ビットとエンタングルさせ、補助ビットの $|0 \rangle$ と $|1 \rangle$ 間の位相を測定する。
$$
e^{−iH \tau} |\phi \rangle |+^{\otimes t} \rangle \to \sum_n e^{−i E_n \tau} c_n |n \rangle |\tilde{E_n} \rangle
$$
測定が行われると、状態が $|c_n|^2 = |\langle n |\phi \rangle|^2$ の確率で $|n \rangle $ に収縮するため、$1/2 < |c_n|^2 \leq 1$ の時は多数決法によって固有値 $E_n$ を求めることができる。


基底エネルギー計算に必要なステップは以下の通りである:
1. ハミルトニアンのサイズを対称性などを用いて削減する
2. ハミルトニアンの時間発展演算子を精度よく近似する ($U^{2k}$ に相当)
3. 制御ユニタリ演算 ($\Lambda (U^{2k})$) を実機で操作可能なゲートセットに分解し実装する
4. 基底状態と十分重なりのある状態を準備する
5. 反復的位相推定法で位相を測定する

以下順を追って手法の詳細な説明と実装例を示す。

#### 7-2-1. ハミルトニアンのサイズを対称性などを用いて削減する
水素分子の第二量子化されたハミルトニアン (STO-6G 基底) を Bravyi-Kitaev 変換した後、電子数保存などの対称性を用いて系の次元を削減すると、電子状態は 2量子ビットで表される (詳しくは文献 [1] を参照されたい)。
$$
H = \sum_i g_i H_i = g_0 I + g_1 Z_0 + g_2 Z_1 + g_3 Z_0 Z_1 + g_4 Y_0 Y_1 + g_5 X_0 X_1
$$
係数 $g_i$ は実数で値は原子間距離に依存する。

#### 7-2-2. ハミルトニアンの時間発展演算子を精度よく近似する
制御ユニタリ演算を行うため、時間発展演算子 $e^{−iH \tau}$ を $\exp(i \theta P)$ の積の形に近似する。ただしここで $P$ はパウリ行列の積である。
`Baker-Campbell-Hausdorff 関係式` を用いるとハミルトニアンの各項 $H_i$ と交換する項 ($I$ と $Z_0 Z_1$) は
$$
e^{−iH \tau} = \exp \left[−i \tau \sum_i g_i H_i \right] = \exp[−i \tau g_0 I] \exp[−i \tau g_3 Z_0 Z_1] \exp[−i \tau H_{\textrm{eff}}]
$$
と積の形で書ける。ただし、
$$
H_{\textrm{eff}} = g_1 Z_0 + g_2 Z_1 + g_4 Y_0 Y_1 + g_5 X_0 X_1
$$
必要なゲートを削減させるため、これらの項を $e^{−iH \tau}$ から除き演算子を簡単化する。これらの固有値への寄与は後から加算する。

一般に $n$ 量子ビットにかかるハミルトニアンの時間発展シミュレーションは、発展時間を $\tau$、必要な精度を $\epsilon$ とすると、ゲート数 $\mathrm{poly} (n, \tau, 1/\epsilon)$ でこれを実現する量子回路 $U_H$ を $||U_H - e^{i H t}|| < \epsilon$ の精度で構築する事が可能である[6]。

本節では Lloyd の方法でハミルトニアンの時間発展を行う。
上式の $H_{\textrm{eff}}$ を Trotter-Suzuki 展開すると、
$$
e^{−iH \tau} = \exp \left[−i \tau \sum_i g_i H_i \right] \approx U_{Torrot} (\tau) = \left( \prod_i \exp[-i g_i H_i \tau/n] \right)^n
$$
が得られる。この近似ではエラーは $n$ を十分大きくとると、
$$
e^{−iH \tau} - \left( \prod_i \exp[-i g_i H_i \tau/n] \right)^n \approx \frac{\tau^2}{2n} \sum_{i > j} [H_i, H_j]
$$

と評価できる。これから $n \approx \tau^2/\epsilon$ 程度の $n$ をとれば良い事が分かる。

今回取り扱う系は $4 \times 4$ の行列で表されるので、参考の為に古典対角化して正確な基底エネルギーを求めてみる。

In [2]:
from functools import reduce
import numpy as np
from numpy.linalg import matrix_power, eig
from scipy.sparse.linalg import eigsh
from openfermion.ops import QubitOperator
from openfermion.transforms import get_sparse_operator
from qulacs import QuantumState, Observable, QuantumCircuit

In [1]:
# Google Colaboratory上でのみ実行してください
from IPython.display import HTML
def setup_mathjax():
    display(HTML('''
    <script>
        if (!window.MathJax) {
            window.MathJax = {
                'tex2jax': {
                    'inlineMath': [['$', '$'], ['\\(', '\\)']],
                    'displayMath': [['$$', '$$'], ['\\[', '\\]']],
                    'processEscapes': true,
                    'processEnvironments': true,
                    'skipTags': ['script', 'noscript', 'style', 'textarea', 'code'],
                    'displayAlign': 'center',
                },
                'HTML-CSS': {
                    'styles': {'.MathJax_Display': {'margin': 0}},
                    'linebreaks': {'automatic': true},
                    // Disable to prevent OTF font loading, which aren't part of our
                    // distribution.
                    'imageFont': null,
                },
               'messageStyle': 'none'
            };
            var script = document.createElement("script");
            script.src = "https://colab.research.google.com/static/mathjax/MathJax.js?config=TeX-AMS_HTML-full,Safe";
            document.head.appendChild(script);
        }
    </script>
    '''))
get_ipython().events.register('pre_run_cell', setup_mathjax)

In [3]:
def reduced_term_hamiltonian():
    """
    distance = 0.70 A
    removed 'I' and 'Z0 Z1' terms, which add up to -1.31916027
    """
    n_qubits = 2
    g_list = [0.3593, 0.0896, -0.4826, 0.0896]
    pauli_strings = ['Z0', 'Y0 Y1', 'Z1', 'X0 X1']
    hamiltonian = QubitOperator()
    for g, h in zip(g_list, pauli_strings):
        hamiltonian += g * QubitOperator(h)
    sparse_matrix = get_sparse_operator(hamiltonian, n_qubits=n_qubits)
    vals, vecs = eigsh(sparse_matrix, k=1, which='SA')
    return sparse_matrix, vals

In [6]:
_, eigs = reduced_term_hamiltonian()
exact_eigenvalue = eigs[0]
print('exact_eigenvalue: {} Ha'.format(exact_eigenvalue))

exact_eigenvalue: -0.860760274408617 Ha


位相推定によるエネルギー測定の精度は、時間発展演算子の近似精度に大きく依存する。水素分子の例では何次の Trotter-Suzuki 展開を行えば十分であろうか？以下では、$H_i^2 = I$ の時
$$
\left( \prod_i \exp[-i g_i H_i \tau/n] \right)^n = \left( \prod_i \cos(g_i\tau/n) I -i \sin(g_i\tau/n) H_i \right)^n
$$
となる性質を用いて $n = 1, 3, \ldots, 9$ において Trotter-Suzuki 展開がどのくらいの精度で演算子を近似できるかを計算するコードを実装した。

In [4]:
def order_n_trotter_suzuki_approx(t, n_trotter_steps):
    """
    ordering: 'Z0', 'Y0 Y1', 'Z1', 'X0 X1'
    Returns:
        sparse_matrix: trotterized [exp(iHt/n)]^n
        args: list of phases of each eigenvalue, exp(i*phase)
    """
    n_qubits = 2
    g_list = [0.3593, 0.0896, -0.4826, 0.0896]
    pauli_strings = ['Z0', 'Y0 Y1', 'Z1', 'X0 X1']
    terms = []
    for g, h in zip(g_list, pauli_strings):
        arg = g * t / n_trotter_steps
        qop = complex(np.cos(arg), 0) * QubitOperator('') - complex(0, np.sin(arg)) * QubitOperator(h)
        terms += [get_sparse_operator(qop, n_qubits=n_qubits)]
    sparse_matrix = reduce(np.dot, terms)
    matrix = matrix_power(sparse_matrix.toarray(), n_trotter_steps)
    vals, vecs = eig(matrix)
    args = np.angle(vals)
    return sparse_matrix, sorted(args)

In [8]:
t = 0.640
print('n, e_trotter, |exact_eigenvalue-e_trotter|')
for n in range(1, 10, 2):
    _, phases = order_n_trotter_suzuki_approx(t, n)
    e_trotter = phases[0]/t
    print(n, e_trotter, abs(exact_eigenvalue-e_trotter)) 

n, e_trotter, |exact_eigenvalue-e_trotter|
1 -0.8602760325707504 0.0004842418378666613
3 -0.860706856078986 5.341832963101645e-05
5 -0.8607410547561056 1.9219652511393015e-05
7 -0.8607504699997903 9.804408826696864e-06
9 -0.8607543437287754 5.930679841670283e-06


お分かり頂けただろうか？ 次数 $n$ が増えるごとに近似精度が上がるが、今回の場合では chemical accuracy( $1.6 × 10^{−3}$ Ha) の精度で近似するには $n = 1$ で十分であることが分かる。

#### 7-2-3. 制御ユニタリ演算を実機で操作可能なゲートセットに分解し実装する
量子コンピュータデバイス上で時間発展演算子 ($\Lambda(U_{Torrot} (2^k t))$) を準備するためには、量子ゲートに分解する必要がある。
今回の例で必要な制御ユニタリゲートは 
* $\Lambda(R_Z(\theta))$
* $\Lambda(R_{XX}(\theta))$
* $\Lambda(R_{YY}(\theta))$ 

である。これらのゲートは全て $\Lambda(R_Z(\theta))$ を用いれば実現可能なので、$\Lambda(R_Z(\theta))$ の実装について詳細に解説する。

先ず $\Lambda(R_Z(\theta))$ は 制御ビット $| c \rangle$ と標的ビット $| t \rangle$ にかかると、
$$
\Lambda(R_Z(\theta)) | c \rangle | t \rangle = | c \rangle R^c_Z(\theta) | t \rangle
$$
を満たすゲートである。
これは $\textrm{CNOT} | c \rangle | t \rangle = | c \rangle X^c | t \rangle$、 $XZX = -Z$ の性質を利用して、
$$
\textrm{CNOT} \left(I \otimes R_Z(-\theta/2) \right) \textrm{CNOT} \left(I \otimes R_Z(\theta/2) \right) | c \rangle | t \rangle
= | c \rangle　X^c　R_Z(-\theta/2) X^c R_Z(\theta/2) | t \rangle
= | c \rangle R^c_Z(\theta) | t \rangle
$$
となり、$\Lambda(R_Z(\theta))$ を実現している。
また、これと
$$
\textrm{CNOT} \left(I \otimes Z_2 \right) \textrm{CNOT} = Z_1 \otimes Z_2
$$
の性質を用いると、$\Lambda(R_{ZZ}(\theta))$ が実現できる。
さらに、$H Z H = X$　や $SH Z HS^{\dagger} = Y$ を用いると $\Lambda(R_{XX}(\theta))$、$\Lambda(R_{YY}(\theta))$ がそれぞれ実現できる。

以下のコードでは `Qulacs` で $\Lambda(U_{Torrot} (2^k t)) = \Lambda(U^{2^k }_{Torrot}(t))$ の量子回路を実装している。
補助ビットに $H$ や $R_Z(\Phi)$ をかけているのは位相推定の為である。

In [5]:
def time_evolution_circuit_exp(g_list, t, kickback_phase, k, n_trotter_step=1):
    n_qubits = 3
    a_idx = 2
    phi = -(t / n_trotter_step) * g_list
    circuit = QuantumCircuit(n_qubits)
    circuit.add_H_gate(a_idx)
    # Apply kickback phase rotation to ancilla bit
    circuit.add_RZ_gate(a_idx, -np.pi*kickback_phase/2)
    for _ in range(n_trotter_step):
        for _ in range(2 ** k):
            # CU(Z0)
            circuit.add_RZ_gate(0, -phi[0])
            circuit.add_CNOT_gate(a_idx, 0)
            circuit.add_RZ_gate(0, phi[0])
            circuit.add_CNOT_gate(a_idx, 0)
        
            # CU(Y0 Y1)
            circuit.add_S_gate(0)
            circuit.add_S_gate(1)
            circuit.add_H_gate(0)
            circuit.add_H_gate(1)
            circuit.add_CNOT_gate(1, 0)
            circuit.add_RZ_gate(0, -phi[1])
            circuit.add_CNOT_gate(a_idx, 0)
            circuit.add_RZ_gate(0, phi[1])
            circuit.add_CNOT_gate(a_idx, 0)
            circuit.add_CNOT_gate(1, 0)                
            circuit.add_H_gate(0)
            circuit.add_H_gate(1)
            circuit.add_Sdag_gate(0)
            circuit.add_Sdag_gate(1)
        
            # CU(Z1)
            circuit.add_RZ_gate(1, -phi[2])
            circuit.add_CNOT_gate(a_idx, 1)
            circuit.add_RZ_gate(1, phi[2])
            circuit.add_CNOT_gate(a_idx, 1)
        
            # CU(X0 X1)
            circuit.add_H_gate(0)
            circuit.add_H_gate(1)
            circuit.add_CNOT_gate(1, 0)
            circuit.add_RZ_gate(0, -phi[3])
            circuit.add_CNOT_gate(a_idx, 0)
            circuit.add_RZ_gate(0, phi[3])
            circuit.add_CNOT_gate(a_idx, 0)
            circuit.add_CNOT_gate(1, 0)     
            circuit.add_H_gate(0)
            circuit.add_H_gate(1)
        
    circuit.add_H_gate(a_idx)
    return circuit

#### 7-2-4. 基底状態と十分重なりのある状態を準備する
今回の水素分子の基底エネルギーの場合では Hartree-Fock (HF) 状態 $|\phi \rangle = |01 \rangle $ が十分基底固有状態に近い為、これを参照状態とする。

#### 7-2-5. 反復的位相推定法で位相を測定する
位相推定アルゴリズムについては既に説明があるのでここでは割愛する。
`Qulacs` では補助ビットなど特定の量子ビットのみを測定しその結果を用いる際に、`state.get_marginal_probability(bit_list)` が便利である。これは量子状態 `state` の特定の量子ビットが特定のビット値を持っている確率を、波動関数の振幅から計算する関数である。
例えば以下のコードでは、補助ビット (`index=2`) が `0` 状態 (0、1番目の量子ビットに関しては測定しない) の確率は、`get_marginal_probability([2, 2, 0])` で得られる (`2` は測定しない事を表している)。

In [6]:
def iterative_phase_estimation(g_list, t, n_itter, init_state, n_trotter_step=1, kickback_phase=0.0):
    for k in reversed(range(1, n_itter)):
        psi = init_state.copy()
        phi = kickback_phase/2
        g_k_list = np.array(g_list)
        circuit = time_evolution_circuit_exp(g_k_list, t, kickback_phase, k, n_trotter_step=n_trotter_step)
        circuit.update_quantum_state(psi)
        # partial trace
        p0 = psi.get_marginal_probability([2, 2, 0])
        p1 = psi.get_marginal_probability([2, 2, 1])
        # update kickback phase
        kth_digit = 1 if (p0 < p1) else 0
        kickback_phase = kickback_phase/2 + kth_digit
    return -0.5 * np.pi * kickback_phase

反復的位相推定法で固有エネルギーを測定するコードは全て揃ったので、実際にシミュレーションを実行してみよう。
ここで、位相を何桁まで測定すれば良いか ($0.j_1 \ldots j_t$ の $t$ をどこまで大きくとるべきか) という疑問がある。
一般に、chemical accuracy($1.6 \times 10^{-3}$ Ha) の精度が必要な場合、イテレーションの回数は[2]
$$
t = - \log_2 (1.6 \times 10^{-3}) + \log_2 \left(2 + \frac{1}{2 \epsilon} \right) \approx 10.87
$$
つまり $t = 11$ 程でとれば十分そうである。実際に $t$ を増加して必要な精度が得られるかどうか見てみる。

In [13]:
n_qubits = 3 # 2 for electron configurations and 1 for ancilla
g_list = [0.3593, 0.0896, -0.4826, 0.0896]
# pauli_strings = ['Z 0', 'Y 0 Y 1', 'Z 1', 'X 0 X 1']
hf_state = QuantumState(n_qubits)
hf_state.set_computational_basis(0b001) # |0>|01>
t = 0.640
n_itter = 12 # determines precission
iqpe_phase = iterative_phase_estimation(g_list, t, n_itter, hf_state, n_trotter_step=1, kickback_phase=0.0)
e_iqpe = iqpe_phase/t
e_trotter = -0.8602760325707504
print('e_iqpe = {} Ha, |e_iqpe-e_trotter| = {} Ha'.format(e_iqpe, abs(e_iqpe-e_trotter)))

e_iqpe = -0.8604673482046018 Ha, |e_iqpe-e_trotter| = 0.00019131563385144101 Ha


お分かり頂けただろうか？ 実際には `n_itter = 12` でようやく chemical accuracy に到達した。

ここで1点注意が必要なのは、ここで紹介したサンプルコードでは $\Lambda(U_{\textrm{trott}}(2^k \tau))$ の深さが $t$ に関して指数的に増大している事である。このハミルトニアンシミュレーションを多項式個のゲートで実現させるための研究もなされている。興味を持たれた読者は文献[3][4][5]を参照されたい。

### 参考文献
[1] Scalable Quantum Simulation of Molecular Energies,
P. J. J. O’Malley et. al.,
PHYSICAL REVIEW X 6, 031007 (2016) <br>
[2] Quantum Computation and Quantum Information,
M. Nielsen and I. Chuang <br>
[3] Efficient quantum algorithms for simulating sparse Hamiltonians, 
D. W. Berry, G. Ahokas, R. Cleve, B. C. Sanders, 
Communications in Mathematical Physics 270, 359 (2007) <br>
[4] Black-box Hamiltonian simulation and unitary implementation, 
D. W. Berry,  A. M. Childs, 
Quantum Information and Computation 12, 29 (2012) <br>
[5] Simulating Hamiltonian dynamics with a truncated Taylor series, 
D. W. Berry, A. M. Childs, R. Cleve, R. Kothari, R. D. Somma, 
Phys. Rev. Lett. 114, 090502 (2015) <br>
[6] Universal Quantum Simulators, S. Lloyd, Science  23 Aug 1996: Vol. 273, Issue 5278, pp. 1073-1078